In [154]:
from dimod import (
    Binary,
    BinaryQuadraticModel,
    ConstrainedQuadraticModel,
    quicksum,
)

from dwave.system import LeapHybridCQMSampler

import random
import pandas as pd
import numpy as np

First, consider the case where there is a single airport where medication can be delivered and the medication must be delivered at the lowest cost possible by a fleet of delivery vehicles. These vehicles are limited in capacity and how far they can travel in one day. To solve this problem efficiently with VQE, we first formalize it as a constrained optimization problem:

Consider when there are $M$ vehicles and $N$ delivery locations. 

Let $x_{i, j, k}$ be an indicator variable such that $x_{i, j, k} = 1$ if vehicle $i$ visits dropoff location $j$ at stop number $k$ on its route and $x_{i, j, k} = 0$ otherwise.  

In [155]:
def build_vrp_cqm(num_locations, distances, num_vehicles, max_distance):
    """
    Input:
        num_locations: number of distinct locations for orders
        distances: an matrix with the distances between respective locations for a fully connected graph
        num_vehicles: the number of vehicles used
        max_distance: the maximum distance that a truck can drive in one day
    """

    M = num_vehicles
    N = num_locations

    cqm = ConstrainedQuadraticModel()

    # Create all the variables: one for each vehicle/location/position combo
    # k is position, j is vertex, i is vehicle
    x = {(i, j, k): Binary(str(i) + "_" + str(j) + "_" + str(k)) for k in range(N+1) for j in range(N+1) for i in range(M)}

    print(f"{x}\n")

    # Define the unconstrained binary optimization problem
    obj = BinaryQuadraticModel(vartype="BINARY")

    # The cost of going from the depot to the first stop
    for m in range(M):
        for n in range(N):
            obj += x[m, n, 0] * distances[N][n]

    # The cost of going from the last stop to the depot
    for m in range(M):
        for n in range(N):
            obj += x[m, n, N-1] * distances[n][N]

    # The cost of going between all stops in the middle
    for m in range(M):
        for n in range(N-1):
            for i in range(N+1):
                for j in range(N+1):
                    obj += x[m, i, n] * x[m, j, n + 1] * distances[i][j]

    cqm.set_objective(obj)

    # Implement constraints:

    # 1. Each location should be served by exactly one vehicle
    for j in range(N):
        sum = 0
        for m in range(M):
            for k in range(N+1):
                sum += x[m, j, k]

        cqm.add_constraint(sum == 1,
                           label=f"Vertex {j} is not visited or visited more than once")

    # 2. Each vehicle is in one place at one time
    for i in range(M):
        for k in range(N+1):

            sum = 0

            for j in range(N + 1):
                sum += x[i, j, k]
            
            cqm.add_constraint(sum == 1,
                               label=f"Vehicle {i} is at more or less than one position at time {k}")
            
    #3. Each vehicle drives less than the cap
    for i in range(M):
        sum = 0

        for j1 in range(N+1):
            for j2 in range(N+1):
                for k in range(N):
                    sum += x[i, j1, k]*x[i, j2, k+1]*distances[j1][j2]

        cqm.add_constraint(sum <= max_distance,
                            label=f"Vehicle {i} drives more than the maximum capacity")

            
    # Return the constrained optimization solution
    return cqm

In [156]:
def run_cqm(cqm):
    """Run the provided CQM on the Leap Hybrid CQM Sampler."""
    sampler = LeapHybridCQMSampler(token="DEV-7a13065bb40813db4f53d6af8d005e3598588034")

    sampleset = sampler.sample_cqm(cqm)
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    num_feasible = len(feasible_sampleset)
    errors = " "
    if num_feasible == 0:
        print("No feasible solution found.")
        return sampleset

    print("\nFeasible solution found.\n")

    return feasible_sampleset

In [157]:
# Create a sample problem
num_destinations = 10
num_vehicles = 2
max_distance = 50

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(6,10)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

cqm = build_vrp_cqm(num_destinations, cost_matrix, num_vehicles, max_distance)

{(0, 0, 0): BinaryQuadraticModel({'0_0_0': 1.0}, {}, 0.0, 'BINARY'), (1, 0, 0): BinaryQuadraticModel({'1_0_0': 1.0}, {}, 0.0, 'BINARY'), (0, 1, 0): BinaryQuadraticModel({'0_1_0': 1.0}, {}, 0.0, 'BINARY'), (1, 1, 0): BinaryQuadraticModel({'1_1_0': 1.0}, {}, 0.0, 'BINARY'), (0, 2, 0): BinaryQuadraticModel({'0_2_0': 1.0}, {}, 0.0, 'BINARY'), (1, 2, 0): BinaryQuadraticModel({'1_2_0': 1.0}, {}, 0.0, 'BINARY'), (0, 3, 0): BinaryQuadraticModel({'0_3_0': 1.0}, {}, 0.0, 'BINARY'), (1, 3, 0): BinaryQuadraticModel({'1_3_0': 1.0}, {}, 0.0, 'BINARY'), (0, 4, 0): BinaryQuadraticModel({'0_4_0': 1.0}, {}, 0.0, 'BINARY'), (1, 4, 0): BinaryQuadraticModel({'1_4_0': 1.0}, {}, 0.0, 'BINARY'), (0, 5, 0): BinaryQuadraticModel({'0_5_0': 1.0}, {}, 0.0, 'BINARY'), (1, 5, 0): BinaryQuadraticModel({'1_5_0': 1.0}, {}, 0.0, 'BINARY'), (0, 6, 0): BinaryQuadraticModel({'0_6_0': 1.0}, {}, 0.0, 'BINARY'), (1, 6, 0): BinaryQuadraticModel({'1_6_0': 1.0}, {}, 0.0, 'BINARY'), (0, 7, 0): BinaryQuadraticModel({'0_7_0': 1.0},

In [159]:
feasible_sampleset = run_cqm(cqm)

KeyboardInterrupt: 

In [ ]:
def parse_string(input_string):
    return list(map(int, input_string.split('_')))

def build_routes_from_sample(sample, num_vehicles):
    """Builds a set of routes from the sample returned."""

    routes =  [[] for _ in range(num_vehicles)]

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            if len(routes[vehicle]) < 1 or vertex != routes[vehicle][-1]:
                routes[vehicle].append(vertex)

    return routes

def build_routes_from_sample_raw(sample, num_vehicles):
    """Builds a set of routes from the sample returned."""

    routes =  [[] for _ in range(num_vehicles)]

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            routes[vehicle].append(vertex)

    return routes

def get_cost_from_sample(paths, distances):
    cost = 0

    for path in paths:
        if len(path) >= 2:
            for i in range(len(path) - 1):
                cost += distances[path[i]][path[i+1]]

    return cost
            



In [ ]:
lowest_energy_sample = feasible_sampleset.lowest().first.sample

routes = build_routes_from_sample(lowest_energy_sample, num_vehicles)
raw_routes = build_routes_from_sample_raw(lowest_energy_sample, num_vehicles)

print(routes)
print(raw_routes)
print(feasible_sampleset.lowest().first.energy)

cost = get_cost_from_sample(routes, cost_matrix)
print(cost)

#print(feasible_sampleset)

[[0, 10, 4], [10, 1, 2, 3, 5, 6, 7, 8, 9]]
[[0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 4], [10, 10, 10, 1, 2, 3, 5, 6, 7, 8, 9]]
64.0
   0_0_0 0_0_1 0_0_10 0_0_2 0_0_3 0_0_4 0_0_5 ... 1_9_9 energy num_oc. ...
0    1.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
1    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
2    0.0   0.0    1.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
3    0.0   0.0    1.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
4    1.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
9    0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
12   1.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
13   0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
14   0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
15   0.0   0.0    0.0   0.0   0.0   0.0   0.0 ...   0.0   64.0       1 ...
17   0.0   0.0    1.0   0.0   0.0   0.0   0.

In [ ]:
min_cost = None
min_path = []

# A very quick and dirty benchmark
for i in range(100):
    array = list(range(num_destinations))
    values = np.random.permutation(array)
    split = random.randint(0, num_destinations)
    path = [values[:split], values[split:]]

    cost = get_cost_from_sample(path, cost_matrix)

    if not min_cost or cost < min_cost:
        min_cost = cost
        min_path = path

print(min_cost)
print(min_path)

57
[array([5, 4, 3, 7, 2, 9, 1]), array([6, 8, 0])]
